In [12]:
import pandas as pd
# transormers
from transformers import BertTokenizer, BertModel
from transformers import AlbertTokenizer, AlbertModel
from transformers import RobertaTokenizer, RobertaModel
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import ElectraTokenizer, ElectraModel
from transformers import XLNetTokenizer, XLNetModel
from gensim.models import Word2Vec
from transformers import BertTokenizer,BertForSequenceClassification,AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer
#torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

In [13]:
df=pd.read_json('data/processed_all.jsonl', lines=True)

---
#### word/sentence embeddings (from e.g. bert-base encoder).

In [3]:
df_features = {'BERT': [], 'ALBERT': [], 'RoBERTa': [] ,'DistilBERT': [], 'XLNet': [],'Electra' : [],
               'word2vec': [] ,  'GloVe': [] , 'tf-idf' : []
              }

In [4]:
embedings = {}
vocab_sizes = {}

In [5]:
def bert_family_embeddings(name, tokenizer, model):
    # tokenize all the texts and the convert tokens to ids
    tokenized_texts = tokenizer(list(df['processed_message']), return_tensors='pt', padding='max_length', truncation=True, max_length = 128)
    # pass the tokenized texts through the  model
    with torch.no_grad():
        outputs = model(**tokenized_texts)
    # extract the embeddings (last layer's hidden states)
    model_embeddings = outputs.last_hidden_state
    vocab_size = tokenizer.vocab_size
    return model_embeddings , vocab_size

---
#### BERT

In [11]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
embedings['BERT'] , vocab_sizes['BERT'] = bert_family_embeddings('BERT', tokenizer, model)

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 13596622848 bytes.

In [ ]:
print(" BERT Vocabulary Size:", vocab_sizes['BERT'])

---
#### ALBERT

In [ ]:
model_name = 'albert-base-v2'
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertModel.from_pretrained(model_name)
embedings['ALBERT'] , vocab_sizes['ALBERT'] = bert_family_embeddings('ALBERT', tokenizer, model)

---
#### RoBERTa

In [ ]:
model_name = "roberta-base" 
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)
embedings['RoBERTa'] , vocab_sizes['RoBERTa'] = bert_family_embeddings('RoBERTa', tokenizer, model)

---
#### DistilBERT

In [ ]:
model_name = "distilbert-base-uncased" 
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)
embedings['DistilBERT'] , vocab_sizes['DistilBERT'] = bert_family_embeddings('DistilBERT', tokenizer, model)

---
#### Electra

In [ ]:
model_name = "google/electra-small-discriminator"  
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraModel.from_pretrained(model_name)
embedings['Electra'] , vocab_sizes['Electra'] = bert_family_embeddings('Electra', tokenizer, model)

---
#### XLNet

In [ ]:
model_name = "xlnet-base-cased"  
tokenizer = XLNetTokenizer.from_pretrained(model_name)
model = XLNetModel.from_pretrained(model_name)
embedings['XLNet'] , vocab_sizes['XLNet'] = bert_family_embeddings('XLNet', tokenizer, model)

---
#### Word2Vec

In [ ]:
# creating word2Vec training dataset.
Embedding_dimensions = 100

# creating Word2Vec training dataset.
Word2vec_train_data = list(map(lambda x: x.split(), df['processed_message']))
# Defining the model and training it.
word2vec_model = Word2Vec( Word2vec_train_data,
                 vector_size=Embedding_dimensions,
                 workers=8,
                 min_count=5)

# Defining the model input length.
input_length = 60
vocab_length = 60000

#X_data, y_data = np.array(x, y)

#tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
#tokenizer.fit_on_texts(X_data)
#tokenizer.num_words = vocab_length
#print("Tokenizer vocab length:", vocab_length)

word2vec_embeddins = pad_sequences(tokenizer.texts_to_sequences(df['processed_message']), maxlen=input_length)
embedings['word2vec'] = word2vec_embeddins
print("word2vec_embeddins.shape:", word2vec_embeddins.shape)

In [ ]:
vocab_sizes['word2vec'] = vocab_length
print("Vocabulary Length:", len(word2vec_model.wv.key_to_index))

---
#### Glove


In [ ]:
url = "http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip"  
filename = "./glove.6B/glove.6B.100d.txt" 

# Check if the file already exists
if not os.path.exists(filename):
    print("File not found. Downloading...")
    urllib.request.urlretrieve(url, filename)
    print("File downloaded.")
    zip_file_path = "glove.6B.zip"  
    output_folder = "glove.6B" 
    # extract the zip archive
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(output_folder)

    print("Files extracted.")  
else:
    print("File already exists.")

In [ ]:
# Load pre-trained GloVe embeddings
glove_file = "./glove.6B/glove.6B.100d.txt"  # path to GloVe file
embeddings_dict = {}
with open(glove_file, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype="float32")
        embeddings_dict[word] = embedding

# Generate document embeddings using GloVe
def generate_embedding(text):
    words = text.split()
    embeddings = [embeddings_dict.get(word, np.zeros(100)) for word in words]
    return np.mean(embeddings, axis=0)

glove_embeddings = df['processed_message'].apply(generate_embedding)
embedings['GloVe'] = glove_embeddings

In [ ]:
vocab_sizes['GloVe'] = len(embeddings_dict)
print("Estimated Vocabulary Size:", len(embeddings_dict))

---
#### TF-IDF

In [ ]:
#Tokenization:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processedtext)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

In [ ]:
vocab_sizes['tf-idf'] = vocab_size

In [ ]:
#Vectorize using TF-IDF 
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=50000)
vectoriser.fit(df['processed_message'])
tfidf_embeddings = vectoriser.transform(df['processed_message'])
embedings['tf-idf'] = tfidf_embeddings
print('number of feature_words: ', len(vectoriser.get_feature_names_out()))